In [1]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import datasets
import glob

import json
import numpy as np
import pandas as pd

from tqdm import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()
datasets.set_caching_enabled(False)

/tmp/ipykernel_2877355/847113659.py:16: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  datasets.set_caching_enabled(False)


In [2]:
# Load Model
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B", src_lang='eng_Latn')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Toxigen & Implicit Hate

In [3]:
def translate_text(text):    
    inputs = tokenizer(text, padding='longest', return_tensors="pt").to('cuda')
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["ind_Latn"], max_length=256
    )
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

def translate_dataset(row):
    text = row['text']
    row['text'] = translate_text(text)
    return row

In [4]:
toxigen_data, implicit_hate_data = json.load(open('toxiGen.json','r')), json.load(open('implicitHate.json', 'r'))
toxigen_dset, implicit_hate_dset = datasets.Dataset.from_list(toxigen_data), datasets.Dataset.from_list(implicit_hate_data)

toxigen_dset = toxigen_dset.map(translate_dataset, batched=True, batch_size=128)
implicit_hate_dset = implicit_hate_dset.map(translate_dataset, batched=True, batch_size=128)

json.dump(toxigen_dset.to_list(), open(f'./toxigen_ind.json', 'w'))
json.dump(implicit_hate_dset.to_list(), open('./implicit_hate_ind.json', 'w'))

Map:   0%|          | 0/6514 [00:00<?, ? examples/s]

Map:   0%|          | 0/14200 [00:00<?, ? examples/s]